In [1]:
!pip install datasets
!pip install razdel
!pip install transformers[torch]
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from pathlib import Path

import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

TRAIN_FILE = "in_domain_train.csv"
IN_DOMAIN_DEV_FILE = "in_domain_dev.csv"

def read_splits(*, as_datasets):
    train_df, in_domain_dev_df = map(
        pd.read_csv, (TRAIN_FILE, IN_DOMAIN_DEV_FILE)
    )


    train_df, dev_df = train_test_split(train_df, test_size=0.1)
    test_df = in_domain_dev_df
    if as_datasets:
        train, dev, test = map(Dataset.from_pandas, (train_df, dev_df, test_df))
        return DatasetDict(train=train, dev=dev, test=test)
    else:
        return train_df, dev_df, test_df

In [9]:
import os
from argparse import ArgumentParser
from functools import partial
from shutil import rmtree

import numpy as np
from datasets import load_metric
from razdel import tokenize
from transformers import (
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    T5Tokenizer,
    T5ForConditionalGeneration,
)

ACCURACY = load_metric("accuracy", keep_in_memory=True)
MCC = load_metric("matthews_correlation", keep_in_memory=True)
MODEL_TO_HUB_NAME = {
    "t5-base": "sberbank-ai/ruT5-base",
    "t5-large": "sberbank-ai/ruT5-large",
}

os.environ["TOKENIZERS_PARALLELISM"] = "false"

N_SEEDS = (10,)
N_EPOCHS = 20
LR_VALUES = (1e-4,)
DECAY_VALUES = (1e-4,)
BATCH_SIZES = (128,)

POS_LABEL = "yes"
NEG_LABEL = "no"


def compute_metrics(p, tokenizer):
    string_preds = tokenizer.batch_decode(p.predictions, skip_special_tokens=True)
    int_preds = [1 if prediction == POS_LABEL else 0 for prediction in string_preds]

    labels = np.where(p.label_ids != -100, p.label_ids, tokenizer.pad_token_id)
    string_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    int_labels = []

    for string_label in string_labels:
        if string_label == POS_LABEL:
            int_labels.append(1)
        elif string_label == NEG_LABEL or string_label == "":  # second case accounts for test data
            int_labels.append(0)
        else:
            raise ValueError()

    acc_result = ACCURACY.compute(predictions=int_preds, references=int_labels)
    mcc_result = MCC.compute(predictions=int_preds, references=int_labels)

    result = {"accuracy": acc_result["accuracy"], "mcc": mcc_result["matthews_correlation"]}

    return result


def preprocess_examples(examples, tokenizer):
    result = tokenizer(examples["sentence"], padding=False)

    if "acceptable" in examples:
        label_sequences = []
        for label in examples["acceptable"]:
            if label == 1:
                target_sequence = POS_LABEL
            elif label == 0:
                target_sequence = NEG_LABEL
            else:
                raise ValueError("Unknown class label")
            label_sequences.append(target_sequence)

    else:
        # a hack to avoid the "You have to specify either decoder_input_ids or decoder_inputs_embeds" error
        # for test data
        label_sequences = ["" for _ in examples["sentence"]]

    result["labels"] = tokenizer(label_sequences, padding=False)["input_ids"]
    result["length"] = [len(list(tokenize(sentence))) for sentence in examples["sentence"]]
    return result


def main(model_name):
    tokenizer = T5Tokenizer.from_pretrained(MODEL_TO_HUB_NAME[model_name])

    splits = read_splits(as_datasets=True)

    tokenized_splits = splits.map(
        partial(preprocess_examples, tokenizer=tokenizer),
        batched=True,
        remove_columns=["sentence"],
    )

    data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8)

    # seed, lr, wd, bs
    dev_metrics_per_run = np.empty((len(N_SEEDS), len(LR_VALUES), len(DECAY_VALUES), len(BATCH_SIZES), 2))

    for i, learning_rate in enumerate(LR_VALUES):
        for j, weight_decay in enumerate(DECAY_VALUES):
            for k, batch_size in enumerate(BATCH_SIZES):
                for l, seed in enumerate(N_SEEDS):
                    model = T5ForConditionalGeneration.from_pretrained(MODEL_TO_HUB_NAME[model_name])

                    run_base_dir = f"{model_name}_{learning_rate}_{weight_decay}_{batch_size}"

                    training_args = Seq2SeqTrainingArguments(
                        output_dir=f"checkpoints/{run_base_dir}",
                        overwrite_output_dir=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=batch_size,
                        per_device_eval_batch_size=batch_size,
                        learning_rate=learning_rate,
                        weight_decay=weight_decay,
                        num_train_epochs=N_EPOCHS,
                        lr_scheduler_type="constant",
                        save_strategy="epoch",
                        save_total_limit=1,
                        seed=seed,
                        fp16=True,
                        dataloader_num_workers=4,
                        group_by_length=True,
                        report_to="none",
                        load_best_model_at_end=True,
                        metric_for_best_model="eval_mcc",
                        optim="adafactor",
                        predict_with_generate=True,
                    )

                    trainer = Seq2SeqTrainer(
                        model=model,
                        args=training_args,
                        train_dataset=tokenized_splits["train"],
                        eval_dataset=tokenized_splits["dev"],
                        compute_metrics=partial(compute_metrics, tokenizer=tokenizer),
                        tokenizer=tokenizer,
                        data_collator=data_collator,
                    )

                    train_result = trainer.train()
                    print(f"{run_base_dir}_{seed}")
                    print("train", train_result.metrics)

                    os.makedirs(f"results/{run_base_dir}_{seed}", exist_ok=True)

                    dev_predictions = trainer.predict(
                        test_dataset=tokenized_splits["dev"], metric_key_prefix="test", max_length=10
                    )
                    print("dev", dev_predictions.metrics)
                    dev_metrics_per_run[l, i, j, k] = (
                        dev_predictions.metrics["test_accuracy"],
                        dev_predictions.metrics["test_mcc"],
                    )

                    predictions = trainer.predict(test_dataset=tokenized_splits["test"], max_length=10)

                    string_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

                    int_preds = [1 if prediction == POS_LABEL else 0 for prediction in string_preds]
                    int_preds = np.asarray(int_preds)

                    np.save(f"results/{run_base_dir}_{seed}/preds.npy", int_preds)

                    rmtree(f"checkpoints/{run_base_dir}")

    os.makedirs("results_agg", exist_ok=True)
    np.save(f"results_agg/{model_name}_dev.npy", dev_metrics_per_run)





In [10]:
main("t5-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/7082 [00:00<?, ? examples/s]

Map:   0%|          | 0/787 [00:00<?, ? examples/s]

Map:   0%|          | 0/983 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Accuracy,Mcc
1,No log,0.235941,0.726811,0.000000
2,No log,0.274863,0.726811,0.000000
3,No log,0.256307,0.726811,0.000000
4,No log,0.210676,0.726811,0.000000
5,No log,0.222463,0.736976,0.156422
6,No log,0.202466,0.752224,0.247672
7,No log,0.249138,0.753494,0.260436
8,No log,0.202328,0.771283,0.337242
9,0.423500,0.225061,0.778907,0.365091
10,0.423500,0.244094,0.773825,0.346023


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


t5-base_0.0001_0.0001_128_10
train {'train_runtime': 831.699, 'train_samples_per_second': 170.302, 'train_steps_per_second': 1.347, 'total_flos': 4272600799641600.0, 'train_loss': 0.2268045499920845, 'epoch': 20.0}


dev {'test_loss': 0.2250606119632721, 'test_accuracy': 0.7789072426937739, 'test_mcc': 0.3650911513747506, 'test_runtime': 5.7717, 'test_samples_per_second': 136.356, 'test_steps_per_second': 1.213}
